In [1]:
import pandas as pd
import numpy as np
import math 

df=pd.read_excel('TX+Collated+Input (2).xlsx',sheet_name='Sheet1')
df1=pd.read_excel('TX+Collated+Input (2).xlsx',sheet_name='Sheet2')

df.rename(columns={'Destination City City':'Destination City'},inplace=True)

df['source_to_destination']=df['srcfc']+'->'+df['desfc']

df1['source_to_destination']=df1['srcfc']+'->'+df1['desfc']

grouped=df.groupby('source_to_destination')['Total Truck Cube'].sum().reset_index()

grouped['sum_status']=grouped['Total Truck Cube'].apply(lambda x:'more than 1000' if x>1000 else 'less than 1000')

df=pd.merge(df,grouped[['source_to_destination','sum_status']],on=['source_to_destination'],how='left')

df_11=df.copy()

df['lane_status']=df['source_to_destination'].apply(lambda x:'defect_lanes' if x in df1['source_to_destination'].values else 'valid_lanes')

df_1=df[(df['sum_status']!='more than 1000') & (df['lane_status']!='valid_lanes')]

df=df[(df['sum_status']=='more than 1000') & (df['lane_status']=='valid_lanes')]

sum_by_route=df.groupby('source_to_destination')['Total Truck Cube'].sum()

rounded_values=np.ceil(sum_by_route/1300)

rounded_values_1=np.floor(sum_by_route/1000)

rounded_values_dict=dict(zip(sum_by_route.index,rounded_values))

rounded_values_dict_1=dict(zip(sum_by_route.index,rounded_values_1))

df['rounded_value']=df['source_to_destination'].map(rounded_values_dict)

df['rounded_value_1000']=df['source_to_destination'].map(rounded_values_dict_1)

df['sum_divided_1000']=df['source_to_destination'].map(lambda x:sum_by_route[x]/1000)

df['sum_divided_1300']=df['source_to_destination'].map(lambda x:sum_by_route[x]/1330)

df['sum_per_lane']=df['source_to_destination'].map(lambda x:sum_by_route[x])

df['change_factor']=(df['sum_per_lane']/df['rounded_value'])

df['diff_round']=df['sum_divided_1000']-df['rounded_value_1000']

df['TX_executable_checks']=df['change_factor'].apply(lambda x:'TX_executable' if (x>=1000 and x<=1330) else 'TX_non_executable')
#df_1.to_excel('defect_lanes_1.xlsx')
df2=df[df['TX_executable_checks']=='TX_non_executable']
df2['srcfc--->desfc']=df['srcfc']+'-->'+df['desfc']
#df1.to_excel('TX_non_executable.xlsx')

/var/folders/nv/t6dzl9xx60l8xc7d4zws11100000gr/T/ipykernel_59467/3273032618.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['srcfc--->desfc']=df['srcfc']+'-->'+df['desfc']


In [2]:
df2.head()

,SL No,srcfc,Source City,desfc,Destination City,ASIN,GL_PRODUCT_GROUP,Units,ACU,Total Truck Cube,...,lane_status,rounded_value,rounded_value_1000,sum_divided_1000,sum_divided_1300,sum_per_lane,change_factor,diff_round,TX_executable_checks,srcfc--->desfc
3,4,XWAA,Mumbai,XSAJ,Bangalore,B0BCKPG2DW,196,6,65.000000,390.000000,...,valid_lanes,3.0,2.0,2.800459,2.105608,2800.458643,933.486214,0.800459,TX_non_executable,XWAA-->XSAJ
9,10,SBHF,Bhopal,SLKD,Lucknow,B08W4Y8SHC,265,1,60.000000,60.000000,...,valid_lanes,2.0,1.0,1.467997,1.103757,1467.996868,733.998434,0.467997,TX_non_executable,SBHF-->SLKD
15,16,BLX1,Bangalore,SCJF,Coimbatore,B0C23KZ9BT,265,1,51.165019,51.165019,...,valid_lanes,4.0,3.0,3.949100,2.969248,3949.099887,987.274972,0.949100,TX_non_executable,BLX1-->SCJF
18,19,SDEB,Delhi,SATB,Ludhiana,B0C23KZ9BT,265,1,51.165019,51.165019,...,valid_lanes,3.0,2.0,2.777637,2.088449,2777.637200,925.879067,0.777637,TX_non_executable,SDEB-->SATB
26,27,SGAC,Guwahati,SCCE,Kolkata,B0C23L29Q4,265,1,51.102802,51.102802,...,valid_lanes,2.0,1.0,1.495833,1.124687,1495.833424,747.916712,0.495833,TX_non_executable,SGAC-->SCCE


In [3]:
df2['closest_multiple']=(df2['sum_per_lane']//1330)*1330
df2['diff'] = df2['sum_per_lane'] - df2['closest_multiple']
df2.sort_values(by=['Units'],ascending=False,inplace=True)
#df2['Units/2']=df2['Units']/2
#df2['revised_units']=np.ceil(df2['Units']-(df2['diff']/df2['ACU']))
#df2.to_excel('test_jhonny.xlsx')

df2.head()

/var/folders/nv/t6dzl9xx60l8xc7d4zws11100000gr/T/ipykernel_59467/2473103593.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['closest_multiple']=(df2['sum_per_lane']//1330)*1330
/var/folders/nv/t6dzl9xx60l8xc7d4zws11100000gr/T/ipykernel_59467/2473103593.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['diff'] = df2['sum_per_lane'] - df2['closest_multiple']
/var/folders/nv/t6dzl9xx60l8xc7d4zws11100000gr/T/ipykernel_59467/2473103593.py:3: SettingWithCopyWarning: 
A value is trying to be set on a

,SL No,srcfc,Source City,desfc,Destination City,ASIN,GL_PRODUCT_GROUP,Units,ACU,Total Truck Cube,...,rounded_value_1000,sum_divided_1000,sum_divided_1300,sum_per_lane,change_factor,diff_round,TX_executable_checks,srcfc--->desfc,closest_multiple,diff
4828,4829,BLX1,Bangalore,SDEB,Delhi,B0B3MMYHYW,504,136,5.032339,684.398159,...,1.0,1.956485,1.471041,1956.485040,978.242520,0.956485,TX_non_executable,BLX1-->SDEB,1330.0,626.485040
1708,1709,SHYH,Hyderabad,BLX1,Bangalore,B0BR3V1SQT,265,111,15.356821,1704.607144,...,2.0,2.660344,2.000259,2660.344261,886.781420,0.660344,TX_non_executable,SHYH-->BLX1,2660.0,0.344261
3914,3915,SBOB,Mumbai,SAME,Ahmedabad,B0987V3K22,196,64,7.178765,459.440934,...,2.0,2.652029,1.994007,2652.028732,884.009577,0.652029,TX_non_executable,SBOB-->SAME,1330.0,1322.028732
5822,5823,SBOB,Mumbai,SDEB,Delhi,B0B6F7LX4C,504,57,1.762908,100.485753,...,1.0,1.661587,1.249314,1661.587197,830.793599,0.661587,TX_non_executable,SBOB-->SDEB,1330.0,331.587197
5821,5822,SBOB,Mumbai,SDEB,Delhi,B0B6F7LX4C,504,57,1.762908,100.485753,...,1.0,1.661587,1.249314,1661.587197,830.793599,0.661587,TX_non_executable,SBOB-->SDEB,1330.0,331.587197


In [4]:
df2['srcfc--->desfc'].unique()

array(['BLX1-->SDEB', 'SHYH-->BLX1', 'SBOB-->SAME', 'SBOB-->SDEB',
       'SMAB-->SCJF', 'BLX1-->SCJF', 'SDEB-->SHYB', 'XWAA-->XSAJ',
       'SHYB-->SBOB', 'SHYH-->SCCE', 'SPAB-->SHYB', 'SMAB-->SCCE',
       'SDEB-->SPAB', 'SHYB-->SCCE', 'SPAB-->SDEB', 'SDEB-->SAME',
       'SBHF-->SHYB', 'XWAA-->XNRW', 'SDEB-->SATB', 'SPAB-->SGAC',
       'SBHF-->SLKD', 'SCJF-->SBOB', 'SGAC-->SCCE'], dtype=object)

In [135]:
df=df2[df2['srcfc--->desfc']=='SDEB-->SPAB'].reset_index()

In [136]:
def closest_multiplier(x,y):
    import math
    optimum = x
    multp = 1
    var = y//x
    delta = math.ceil(y/x)-(y/x)
    if delta < 0.01:
        return x*math.ceil(y/x)
    else:
        return x*var

In [137]:
# new algo
x = 1330
r = 9
sum_per_lane = 0
for i in range(len(df)):
    sum_per_lane += df['Units'][i]*df['ACU'][i]
y = sum_per_lane
print("Sum per lane:", sum_per_lane)

Sum per lane: 1713.4749321929996


In [138]:
df.columns

Index(['index', 'SL No', 'srcfc', 'Source City', 'desfc', 'Destination City',
       'ASIN', 'GL_PRODUCT_GROUP', 'Units', 'ACU', 'Total Truck Cube',
       'source_to_destination', 'sum_status', 'lane_status', 'rounded_value',
       'rounded_value_1000', 'sum_divided_1000', 'sum_divided_1300',
       'sum_per_lane', 'change_factor', 'diff_round', 'TX_executable_checks',
       'srcfc--->desfc', 'closest_multiple', 'diff'],
      dtype='object')

In [139]:
df.head()

,index,SL No,srcfc,Source City,desfc,Destination City,ASIN,GL_PRODUCT_GROUP,Units,ACU,...,rounded_value_1000,sum_divided_1000,sum_divided_1300,sum_per_lane,change_factor,diff_round,TX_executable_checks,srcfc--->desfc,closest_multiple,diff
0,1134,1135,SDEB,Delhi,SPAB,Patna,B0BR3YSP3Q,265,18,18.534000,...,1.0,1.713475,1.288327,1713.474932,856.737466,0.713475,TX_non_executable,SDEB-->SPAB,1330.0,383.474932
1,1840,1841,SDEB,Delhi,SPAB,Patna,B0BR3Z1KGN,265,10,14.237361,...,1.0,1.713475,1.288327,1713.474932,856.737466,0.713475,TX_non_executable,SDEB-->SPAB,1330.0,383.474932
2,843,844,SDEB,Delhi,SPAB,Patna,B0BR3Y21DG,265,9,21.955116,...,1.0,1.713475,1.288327,1713.474932,856.737466,0.713475,TX_non_executable,SDEB-->SPAB,1330.0,383.474932
3,1217,1218,SDEB,Delhi,SPAB,Patna,B0BRNX1TMX,265,8,18.200965,...,1.0,1.713475,1.288327,1713.474932,856.737466,0.713475,TX_non_executable,SDEB-->SPAB,1330.0,383.474932
4,4645,4646,SDEB,Delhi,SPAB,Patna,B08427XSCY,196,7,5.848814,...,1.0,1.713475,1.288327,1713.474932,856.737466,0.713475,TX_non_executable,SDEB-->SPAB,1330.0,383.474932


In [140]:
closest_multiplier(x,y)

1330.0

In [141]:
"""
Configurations ~ change here for future
"""
#max value of ACU configured here
max_value = closest_multiplier(x,y)
#sum per lane
delta_val = max_value-y
# minimum 50% count of previous 
min_thresh = 0.5
start_thresh = 1
#minimum quantity assingment
min_qty = 2



In [146]:
import traceback

column_name_reqd = ['SL No', 'srcfc', 'Source City', 'desfc', 'Destination City',
       'ASIN', 'GL_PRODUCT_GROUP', 'Units', 'ACU', 'Total Truck Cube',
       'source_to_destination', 'sum_status', 'lane_status', 'rounded_value',
       'rounded_value_1000', 'sum_divided_1000', 'sum_divided_1300',
       'sum_per_lane', 'change_factor', 'diff_round', 'TX_executable_checks',
       'srcfc--->desfc', 'closest_multiple', 'diff', 'accom_unit',
       'acu_accom_val', 'lane_total_acu']


"""
array tuple acu sum function
"""
def sub_arr_sm(arr,s):
    sum_arr = sum([x[0] for x in arr])
    #check if sum of array is less than equal to s return arr
    if sum_arr <= s:
        return arr
    #else if greater than s, then get max sub array sum less than s
    #for a tie in max sub array sum get the min length of the arr
    else:
        arr.sort()
        result = []
        max_sum = s
        min_sum = 0
        min_len = len(arr)
        i = 0
        j = len(arr)
        while i<j:
            sum_arr = sum([x[0] for x in arr[i:j]])
            len_arr = len(arr[i:j])
            if sum_arr > min_sum and sum_arr <= max_sum and len_arr < min_len:
                result = arr[i:j]
                min_sum = sum_arr
                min_len = len_arr
            j -= 1
            if i == j:
                j = len(arr)
                i += 1
        return result

def get_arr_tup(df_new):
    import json
    arr = json.loads(df_new[['ASIN', 'ACU']].to_json(orient='records'))
    arr_tup = []
    for i in range(len(arr)):
        v = (arr[i]['ACU'],arr[i]['ASIN'])
        arr_tup.append(v)
    return arr_tup

def funct_som(df, max_value, delta_val, min_qty):
    s = delta_val
    #New Quantity
    df['accom_unit']= df['Units']
    #New Quantity* ACU
    df['acu_accom_val']= df['Units']*df['ACU']
    df['lane_total_acu'] = df['acu_accom_val'].sum()
    min_acu_val = df['ACU'].min()
    
    df_new = df.copy(deep=True)
    
    arr_tup = get_arr_tup(df_new)
    
    #print("Decrease ACU left:", s)
    
    """
    run this in loop with delta
    arr_tup == > [('acu','asin')]
    """
    c = 0
    while True:
        #print(df_new)
        min_acu_val = df_new['ACU'].min()
        #arr_tup
        val_to_decrease_qty = sub_arr_sm(arr_tup, s)
        result = val_to_decrease_qty
        
        """
        if no algo optimisation possible
        run loop to get acu sort min to max and run this below logic -- to be done.
        
        """
        if val_to_decrease_qty and len(val_to_decrease_qty) == 0:
            #print("hh")
            len_df = len(df_new)
            i = 0
            while i < len_df:
                asin = df_new['ASIN'][i]
                idxMain = df[ (df['ASIN'] == asin) ].index[0]
                #print(idxMain)
                if df['accom_unit'][idxMain] <= min_qty:
                    #print("Drop h:",asin)
                    idxDrop = df_new[ (df_new['ASIN'] == asin) ].index
                    #print("len pre drop:", len(df_new))
                    df_new.drop(idxDrop , inplace=True)
                    df_new = df_new[column_name_reqd]
                    df_new = df_new.reset_index()
                    len_df = len(df_new)
                    #print("len post drop:", len(df_new))
                    arr_tup = get_arr_tup(df_new)
                    #print(arr_tup)
                    i -= 1
                elif df['accom_unit'][idxMain] > min_qty and df['accom_unit'][idxMain]-1 > math.ceil(df['Units'][idxMain]/2):
                    print('lk')
                    df['accom_unit'][idxMain] -= 1
                i += 1
                #print(df_new)
                #print(arr_tup)
                    
        else:
            #add qty -1 to all asins in result
            #print('hh2:', len(val_to_decrease_qty))
            for val in val_to_decrease_qty:
                asin = val[1]
                # Applying the condition
                idx = df[df["ASIN"] == asin].index[0]
                #print(asin, math.ceil(df['Units'][idx]/2))
                if df['accom_unit'][idx] > min_qty and df['accom_unit'][idx]-1 >= math.ceil(df['Units'][idx]/2):
                    df['accom_unit'][idx] -= 1
                else:
                    #print("Drop:",asin)
                    idxDrop = df_new[ (df_new['ASIN'] == asin) ].index
                    df_new.drop(idxDrop , inplace=True)
                    #print("len pre drop:", len(df_new))
                    df_new = df_new[column_name_reqd]
                    df_new = df_new.reset_index()
                    #print("len post drop:", len(df_new))
                    arr_tup = get_arr_tup(df_new)
                    #print(arr_tup)
            
                
        #get new array tuple for sort
        arr_tup = get_arr_tup(df_new)
        #New Quantity* ACU
        df['acu_accom_val']= df['accom_unit']*df['ACU']
        total_acu = df['acu_accom_val'].sum()
        df['lane_total_acu'] = total_acu
        delta_accom = total_acu-max_value
        #print("Min acu value:", min_acu_val)
        
        if len(df_new) == 0:
            break
        if len(arr_tup) == 0:
            break
        if delta_accom < 0:
            #print('h')
            break
        if delta_accom < min_acu_val:
            break
        else:
            s = delta_accom
        #print("delta left:",delta_accom)
    
    #return final algorithm optimised result
    #print("Returning new algorithm optimised quantity for optimum accomodation")
    
    return df


In [147]:
df_final = funct_som(df, max_value, delta_val, min_qty)

/var/folders/nv/t6dzl9xx60l8xc7d4zws11100000gr/T/ipykernel_59467/1068109381.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['accom_unit'][idx] -= 1
/var/folders/nv/t6dzl9xx60l8xc7d4zws11100000gr/T/ipykernel_59467/1068109381.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['accom_unit'][idx] -= 1
/var/folders/nv/t6dzl9xx60l8xc7d4zws11100000gr/T/ipykernel_59467/1068109381.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

In [148]:
df_final.head()

,index,SL No,srcfc,Source City,desfc,Destination City,ASIN,GL_PRODUCT_GROUP,Units,ACU,...,sum_per_lane,change_factor,diff_round,TX_executable_checks,srcfc--->desfc,closest_multiple,diff,accom_unit,acu_accom_val,lane_total_acu
0,1134,1135,SDEB,Delhi,SPAB,Patna,B0BR3YSP3Q,265,18,18.534000,...,1713.474932,856.737466,0.713475,TX_non_executable,SDEB-->SPAB,1330.0,383.474932,15,278.010000,1333.243082
1,1840,1841,SDEB,Delhi,SPAB,Patna,B0BR3Z1KGN,265,10,14.237361,...,1713.474932,856.737466,0.713475,TX_non_executable,SDEB-->SPAB,1330.0,383.474932,7,99.661526,1333.243082
2,843,844,SDEB,Delhi,SPAB,Patna,B0BR3Y21DG,265,9,21.955116,...,1713.474932,856.737466,0.713475,TX_non_executable,SDEB-->SPAB,1330.0,383.474932,8,175.640924,1333.243082
3,1217,1218,SDEB,Delhi,SPAB,Patna,B0BRNX1TMX,265,8,18.200965,...,1713.474932,856.737466,0.713475,TX_non_executable,SDEB-->SPAB,1330.0,383.474932,5,91.004826,1333.243082
4,4645,4646,SDEB,Delhi,SPAB,Patna,B08427XSCY,196,7,5.848814,...,1713.474932,856.737466,0.713475,TX_non_executable,SDEB-->SPAB,1330.0,383.474932,5,29.244072,1333.243082


In [149]:
"""
accom_unit --> New column with New units for optimum space
lane_total_acu --> Final value of optimum space close multiple of 1330
"""

'\naccom_unit --> New column with New units for optimum space\nlane_total_acu --> Final value of optimum space close multiple of 1330\n'